In [ ]:
import pandas as pd # 표 형식의 데이터를 다룰 수 있는 pandas를 pd라고 줄여서 불러옵니다
from selenium import webdriver # 크롬 창을 조종하기 위한 모듈입니다
from selenium.webdriver.common.by import By # 웹사이트의 구성요소를 선택하기 위해 By 모듈을 불려옵니다
from selenium.webdriver.support.ui import WebDriverWait # 웹페이지가 전부 로드될때까지 기다리는 (Explicitly wait) 기능을 하는 모듈입니다
from webdriver_manager.chrome import ChromeDriverManager # 크롬에서 크롤링을 하기 위해, 웹 드라이버를 설치하는 모듈입니다
from selenium.webdriver.support import expected_conditions as EC # 크롬의 어떤 부분의 상태를 확인하는 모듈입니다
import time # 정해진 시간만큼 기다리게 하기 위한 패키지입니다

In [ ]:
# 웹 드라이버를 설치하고, 조종할 수 있는 크롬 창을 실행합니다
driver = webdriver.Chrome()


In [91]:
driver.get("https://map.naver.com/v5/search/서울 강남구 카페")


In [92]:
try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "input_search"))
    )  # 네이버 지도의 검색창은 "input_search" 라는 클래스 이름으로 설정되어 있습니다
finally:
    pass


In [93]:
driver.switch_to.frame("searchIframe")

fnm = ''  # 맨 첫번째 상호 이름입니다. 다음 페이지에서도 똑같은 상호가 나온다면, 다음 페이지가 없다고 인식하고, brk 값이 바뀌면서 반복문이 종료되지요
brk = 1
res = pd.DataFrame()  # 결과 파일은 판다스 데이터프레임으로 입력할겁니다
# 크롤링할 데이터가 있는 영역 중, 빈 공간을 입력해 뒀습니다
empty = '//*[@id="_pcmap_list_scroll_container"]'

while brk:  # 페이지 설정

    # 이렇게 find_element 함수만 사용해 놓으면 그 영역이 인식되더라고요
    driver.find_element(By.XPATH, empty)

    for i in range(1, 51):  # 1~50번째 상호를 순회하도록 했습니다
        nm = ['NA']  # 상호가 저장될 변수
        place_type = ['NA']  # 장소 종류가 저장될 변수

        driver.find_element(By.XPATH, empty)

        # 네이버 지도는 이름이 두 가지 영역에 저장되는 것을 확인했습니다. 일단 둘 다 긁어오고, 이름이 있는지는 나중에 체크하도록 했습니다
        nm = driver.find_elements(
            By.XPATH, f'//*[@id="_pcmap_list_scroll_container"]/ul/li[{i}]/div[1]/a[1]/div/div/span[1]')
        nm += driver.find_elements(
            By.XPATH, f'//*[@id="_pcmap_list_scroll_container"]/ul/li[{i}]/div[1]/div/a[1]/div/div/span[1]')

        driver.find_element(By.XPATH, empty)

        # 장소 종류도 이름과 마찬가지로 했습니다
        place_type = driver.find_elements(
            By.XPATH, f'//*[@id="_pcmap_list_scroll_container"]/ul/li[{i}]/div[1]/a[1]/div/div/span[2]')
        place_type += driver.find_elements(
            By.XPATH, f'//*[@id="_pcmap_list_scroll_container"]/ul/li[{i}]/div[1]/div/div/div/span/a/span[1]')

        

        if nm != []:  # 이름이 비어있으면 아무것도 안하도록 했습니다
            if "네이버" in place_type[0].text :
                place_type = driver.find_elements(
                    By.XPATH, f'//*[@id="_pcmap_list_scroll_container"]/ul/li[{i}]/div[1]/a[1]/div/div/span[last()]')

            place_type = place_type[0].text

            # 강남구로 검색했음에도 강남구 인접 지역이 검색되는 경우가 있습니다. 이 때는 장소 종류를 확인해 줘야 합니다
            # if any(i in place_type for i in ['강남구']):
            # res 데이터프레임에 차곡차곡 쌓아줍니다
            res = pd.concat([res, pd.DataFrame([nm[0].text, place_type]).T])
            # 데이터가 실시간으로 저장되도록 합니다
            res.to_csv('./res_naver_cafe.csv', index=False)

        if i == 1:  # 첫번째 상호를 불러왔다면, 이전 페이지의 첫번째 상호와 같은지 확인해 줍니다
            if fnm == nm:
                brk = 0
                break

            else:
                fnm = nm

    # 다음 페이지로 넘어가는 코드입니다 다음 버튼을 인식해서 클릭하도록 만들었습니다
    driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div[3]/div[2]')
    driver.find_element(
        By.XPATH, '//*[@id="app-root"]/div/div[3]/div[2]/a[7]').click()
    time.sleep(2)  # 페이지 로딩 시간 2초


In [94]:
res.head()


,0,1
0,트리오드,"카페,디저트"
0,클로리스 코엑스몰점,홍차전문점
0,뉴욕라츠오베이글스,베이글
0,당옥,"카페,디저트"
0,런던 베이글 뮤지엄 도산점,베이커리
